In [10]:
import torch
import torch.nn as nn
from torch.optim import Adam
from models.new_wave import WaveNet
from disk_utils import save_model, load_model
from audio_dataset import build_audio_data_loaders

In [11]:
test_size = 0.0
train_dl, _ = build_audio_data_loaders(test_size=test_size)

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cpu = torch.device("cpu")

model = WaveNet(residual_channels=64,
                skip_channels=128,
                num_blocks=2,
                num_layers=10,
                kernel_size=2)
criterion = nn.MSELoss()
optimizer = Adam(model.parameters(), lr=2e-4)

In [ ]:
num_epochs = 2

In [ ]:
model = model.to(device)
model.train()
for epoch in range(num_epochs):
    running_loss = 0.0
    for X, Y, _, _ in train_dl:
        X = X.to(device)
        Y = Y.to(device)

        # Forward pass
        outputs = model(X)
        loss = criterion(outputs, Y)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(
        f"E {epoch + 1}/{num_epochs}\t Loss: {running_loss / len(train_dl):.6f}")

In [9]:
model.to(cpu)
save_model(model, "wavenet")